#### Here 3 layers MLP is implemented. The data set used here is as same as that  is used for Simple_Perceptron.

In [ ]:
import tensorflow as tf
from util.data import *


##### Data and Parameters ----------------------------------------------------------------

In [ ]:
learnin_rate=0.01
epochs=200
display_step=50
Feature_Number=1024
Sample_Number=10000
batch_size=256

DataObj=Data(data_len=Feature_Number,Sample_Number=Sample_Number,Test_percentage=40)
Train_x,Train_y,Test_x,Test_y=DataObj.CNN_data()
Number_of_batches=int(len(Train_x)/batch_size)

# There is nothing behind the number of hiiden units choosen for the layers
n_hidden1=700
n_hidden2=700
n_hidden3=700
n_o=10 # number of calasses


#### Structure Definition----------------------------------------------------------


In [ ]:
X=tf.placeholder(dtype=tf.float32,shape=[None,Feature_Number])
Y=tf.placeholder(dtype=tf.float32,shape=[None,10])


Weights={'hidden1':tf.Variable(tf.random_normal([Feature_Number,n_hidden1],stddev=0.01)),
        'hidden2':tf.Variable(tf.random_normal([n_hidden1,n_hidden2],stddev=0.01)),
        'hidden3':tf.Variable(tf.random_normal([n_hidden2,n_hidden3],stddev=0.01)),
        'Out':tf.Variable(tf.random_normal([n_hidden3,n_o],stddev=0.01))}

biases={'b1':tf.Variable(tf.zeros([n_hidden1]),dtype=tf.float32),
       'b2':tf.Variable(tf.zeros([n_hidden2]),dtype=tf.float32),
       'b3':tf.Variable(tf.zeros([n_hidden3]),dtype=tf.float32),
       'bo':tf.Variable(tf.zeros([n_o]),dtype=tf.float32)}

def model(X,W,b):
    h1=tf.nn.tanh(tf.add(tf.matmul(X,W['hidden1']),b['b1']))
    h2=tf.nn.relu(tf.add(tf.matmul(h1,W['hidden2']),b['b2']))
    h3=tf.nn.sigmoid(tf.add(tf.matmul(h2,W['hidden3']),b['b3']))
    out=tf.nn.relu(tf.add(tf.matmul(h3,W['Out']),b['bo']))
    results=tf.nn.softmax(out)
    return results

Pred=model(X,Weights,biases)

cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Pred,labels=Y))
train_step=tf.train.GradientDescentOptimizer(learnin_rate).minimize(cost)

Correct_predictions=tf.equal(tf.argmax(Pred,1),tf.argmax(Y,1))
Accuracy=tf.reduce_mean(tf.cast(Correct_predictions,"float"))



#### Graph Running --------------------------------------------------------------------------

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(epochs):
        avg_cost=0
        for j in range(Number_of_batches) :
            Trx,Try=next_batch(batch_size, Train_x,Train_y)
            sess.run(train_step,feed_dict={X:Trx,Y:Try})
            avg_cost+=sess.run(cost,feed_dict={X:Trx,Y:Try})/Number_of_batches
        if i% display_step==0:
            Train_Accuracy=sess.run(Accuracy,feed_dict={X:Train_x,Y:Train_y})
            Test_Accuracy=sess.run(Accuracy,feed_dict={X:Test_x,Y:Test_y})
            print ("Epoch: %03d/%03d cost: %.9f train_acc: %.3f test_acc: %.3f" 
                   % (i, epochs, avg_cost, Train_Accuracy, Test_Accuracy))
sess.close()
# Don't worry if you get low and repetitive accuracy on Test data!!